In [1]:
import os
import sys
sys.path.append(os.chdir('../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from sklearn.multioutput import MultiOutputRegressor
from sklearn.utils import shuffle
from catboost import CatBoostRegressor

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

In [3]:
import src.data as data
import src.utility as utility

In [4]:
#dataset_train = base_df.merge(data.base_dataset(), how='left')
X, Y = data.dataset(onehot=False, drop_index_columns=False)

caching base dataset train


In [5]:
print(X.shape)
print(Y.shape)

(34083, 74)
(34083, 4)


In [19]:
X.dtypes

EMERGENCY_LANE    float64
LANES             float64
ROAD_TYPE         float64
EVENT_DETAIL      float64
EVENT_TYPE        object 
SPEED_AVG_-10     float64
SPEED_AVG_-9      float64
SPEED_AVG_-8      float64
SPEED_AVG_-7      float64
SPEED_AVG_-6      float64
SPEED_AVG_-5      float64
SPEED_AVG_-4      float64
SPEED_AVG_-3      float64
SPEED_AVG_-2      float64
SPEED_AVG_-1      float64
SPEED_SD_-10      float64
SPEED_SD_-9       float64
SPEED_SD_-8       float64
SPEED_SD_-7       float64
SPEED_SD_-6       float64
SPEED_SD_-5       float64
SPEED_SD_-4       float64
SPEED_SD_-3       float64
SPEED_SD_-2       float64
SPEED_SD_-1       float64
SPEED_SD_Y_0      float64
SPEED_SD_Y_1      float64
SPEED_SD_Y_2      float64
SPEED_SD_Y_3      float64
SPEED_MAX_-10     float64
                   ...   
SPEED_MAX_Y_2     float64
SPEED_MAX_Y_3     float64
SPEED_MIN_-10     float64
SPEED_MIN_-9      float64
SPEED_MIN_-8      float64
SPEED_MIN_-7      float64
SPEED_MIN_-6      float64
SPEED_MIN_-5

In [78]:
Y.dtypes

SPEED_AVG_Y_0    float64
SPEED_AVG_Y_1    float64
SPEED_AVG_Y_2    float64
SPEED_AVG_Y_3    float64
dtype: object

## Preprocess data

In [10]:
pd.read_csv('resources/dataset/preprocessed/base_dataframe_train.csv.gz')

,KEY,KM,event_index,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,DATETIME_UTC_-10,DATETIME_UTC_-9,DATETIME_UTC_-8,DATETIME_UTC_-7,DATETIME_UTC_-6,DATETIME_UTC_-5,DATETIME_UTC_-4,DATETIME_UTC_-3,DATETIME_UTC_-2,DATETIME_UTC_-1,DATETIME_UTC_y_0,DATETIME_UTC_y_1,DATETIME_UTC_y_2,DATETIME_UTC_y_3,SPEED_AVG_-10,SPEED_AVG_-9,SPEED_AVG_-8,SPEED_AVG_-7,SPEED_AVG_-6,SPEED_AVG_-5,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_AVG_Y_0,SPEED_AVG_Y_1,SPEED_AVG_Y_2,SPEED_AVG_Y_3,SPEED_SD_-10,SPEED_SD_-9,SPEED_SD_-8,SPEED_SD_-7,SPEED_SD_-6,SPEED_SD_-5,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_SD_Y_0,SPEED_SD_Y_1,SPEED_SD_Y_2,SPEED_SD_Y_3,SPEED_MAX_-10,SPEED_MAX_-9,SPEED_MAX_-8,SPEED_MAX_-7,SPEED_MAX_-6,SPEED_MAX_-5,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MAX_Y_0,SPEED_MAX_Y_1,SPEED_MAX_Y_2,SPEED_MAX_Y_3,SPEED_MIN_-10,SPEED_MIN_-9,SPEED_MIN_-8,SPEED_MIN_-7,SPEED_MIN_-6,SPEED_MIN_-5,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,SPEED_MIN_Y_0,SPEED_MIN_Y_1,SPEED_MIN_Y_2,SPEED_MIN_Y_3,N_VEHICLES_-10,N_VEHICLES_-9,N_VEHICLES_-8,N_VEHICLES_-7,N_VEHICLES_-6,N_VEHICLES_-5,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,N_VEHICLES_Y_0,N_VEHICLES_Y_1,N_VEHICLES_Y_2,N_VEHICLES_Y_3
0,0.0,337.0,2674.0,1.0,2.0,1.0,NaN,NaN,2018-09-05 16:30:00,2018-09-05 16:45:00,2018-09-05 17:00:00,2018-09-05 17:15:00,2018-09-05 17:30:00,2018-09-05 17:45:00,2018-09-05 18:00:00,2018-09-05 18:15:00,2018-09-05 18:30:00,2018-09-05 18:45:00,2018-09-05 19:00:00,2018-09-05 19:15:00,2018-09-05 19:30:00,2018-09-05 19:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199.6642,199.3150,204.7759,201.7721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.4457,52.0643,43.0659,49.9930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263.734,292.858,294.476,312.274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,59.866,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,129.0,139.0,193.0,NaN,NaN,NaN
1,0.0,337.0,2734.0,1.0,2.0,1.0,6.0,Gestione_viabilita,2018-09-20 14:00:00,2018-09-20 14:15:00,2018-09-20 14:30:00,2018-09-20 14:45:00,2018-09-20 15:00:00,2018-09-20 15:15:00,2018-09-20 15:30:00,2018-09-20 15:45:00,2018-09-20 16:00:00,2018-09-20 16:15:00,2018-09-20 16:30:00,2018-09-20 16:45:00,2018-09-20 17:00:00,2018-09-20 17:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,173.4133,175.3361,171.4932,194.6957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.2940,21.7815,33.8448,35.4031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236.228,245.936,237.846,292.858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,63.102,59.866,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214.0,235.0,219.0,148.0
2,0.0,337.0,2795.0,1.0,2.0,1.0,6.0,Gestione_viabilita,2018-10-01 14:45:00,2018-10-01 15:00:00,2018-10-01 15:15:00,2018-10-01 15:30:00,2018-10-01 15:45:00,2018-10-01 16:00:00,2018-10-01 16:15:00,2018-10-01 16:30:00,2018-10-01 16:45:00,2018-10-01 17:00:00,2018-10-01 17:15:00,2018-10-01 17:30:00,2018-10-01 17:45:00,2018-10-01 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.9124,201.9654,215.8426,218.5883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.8807,39.7029,30.8512,28.1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352.724,286.386,343.016,318.746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,61.484,77.664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0,216.0,227.0,184.0,NaN
3,0.0,337.0,2796.0,1.0,2.0,1.0,6.0,Gestione_viabilita,2018-10-01 15:00:00,2018-10-01 15:15:00,2018-10-01 15:30:00,2018-10-01 15:45:00,2018-10-01 16:00:00,2018-10-01 16:15:00,2018-10-01 16:30:00,2018-10-01 16:45:00,2018-10-01 17:00:00,2018-10-01 17:15:00,2018-10-01 17:30:00,2018-10-01 17:45:00,2018-10-01 18:00:00,2018-10-01 18:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.9124,201.9654,215.8426,218.5883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.8807,39.7029,30.8512,28.1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352.724,286.386,343.016,318.746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,61.484,77.664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0,216.0,227.0,184.0,NaN,NaN
4,0.0,337.0,2870.0,1.0,2.0,1.0,6

In [7]:
X[X.event_index.isnull()]

,KEY,KM,event_index,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,SPEED_AVG_-10,SPEED_AVG_-9,SPEED_AVG_-8,SPEED_AVG_-7,SPEED_AVG_-6,SPEED_AVG_-5,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_SD_-10,SPEED_SD_-9,SPEED_SD_-8,SPEED_SD_-7,SPEED_SD_-6,SPEED_SD_-5,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_SD_Y_0,SPEED_SD_Y_1,SPEED_SD_Y_2,SPEED_SD_Y_3,SPEED_MAX_-10,SPEED_MAX_-9,SPEED_MAX_-8,SPEED_MAX_-7,SPEED_MAX_-6,SPEED_MAX_-5,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MAX_Y_0,SPEED_MAX_Y_1,SPEED_MAX_Y_2,SPEED_MAX_Y_3,SPEED_MIN_-10,SPEED_MIN_-9,SPEED_MIN_-8,SPEED_MIN_-7,SPEED_MIN_-6,SPEED_MIN_-5,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,SPEED_MIN_Y_0,SPEED_MIN_Y_1,SPEED_MIN_Y_2,SPEED_MIN_Y_3,N_VEHICLES_-10,N_VEHICLES_-9,N_VEHICLES_-8,N_VEHICLES_-7,N_VEHICLES_-6,N_VEHICLES_-5,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,N_VEHICLES_Y_0,N_VEHICLES_Y_1,N_VEHICLES_Y_2,N_VEHICLES_Y_3


In [9]:
X.columns

Index(['KEY', 'KM', 'event_index', 'EMERGENCY_LANE', 'LANES', 'ROAD_TYPE',
       'EVENT_DETAIL', 'EVENT_TYPE', 'SPEED_AVG_-10', 'SPEED_AVG_-9',
       'SPEED_AVG_-8', 'SPEED_AVG_-7', 'SPEED_AVG_-6', 'SPEED_AVG_-5',
       'SPEED_AVG_-4', 'SPEED_AVG_-3', 'SPEED_AVG_-2', 'SPEED_AVG_-1',
       'SPEED_SD_-10', 'SPEED_SD_-9', 'SPEED_SD_-8', 'SPEED_SD_-7',
       'SPEED_SD_-6', 'SPEED_SD_-5', 'SPEED_SD_-4', 'SPEED_SD_-3',
       'SPEED_SD_-2', 'SPEED_SD_-1', 'SPEED_SD_Y_0', 'SPEED_SD_Y_1',
       'SPEED_SD_Y_2', 'SPEED_SD_Y_3', 'SPEED_MAX_-10', 'SPEED_MAX_-9',
       'SPEED_MAX_-8', 'SPEED_MAX_-7', 'SPEED_MAX_-6', 'SPEED_MAX_-5',
       'SPEED_MAX_-4', 'SPEED_MAX_-3', 'SPEED_MAX_-2', 'SPEED_MAX_-1',
       'SPEED_MAX_Y_0', 'SPEED_MAX_Y_1', 'SPEED_MAX_Y_2', 'SPEED_MAX_Y_3',
       'SPEED_MIN_-10', 'SPEED_MIN_-9', 'SPEED_MIN_-8', 'SPEED_MIN_-7',
       'SPEED_MIN_-6', 'SPEED_MIN_-5', 'SPEED_MIN_-4', 'SPEED_MIN_-3',
       'SPEED_MIN_-2', 'SPEED_MIN_-1', 'SPEED_MIN_Y_0', 'SPEED_MIN_Y_1',
    

In [8]:
X.head(15)

,KEY,KM,event_index,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,SPEED_AVG_-10,SPEED_AVG_-9,SPEED_AVG_-8,SPEED_AVG_-7,SPEED_AVG_-6,SPEED_AVG_-5,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_SD_-10,SPEED_SD_-9,SPEED_SD_-8,SPEED_SD_-7,SPEED_SD_-6,SPEED_SD_-5,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_SD_Y_0,SPEED_SD_Y_1,SPEED_SD_Y_2,SPEED_SD_Y_3,SPEED_MAX_-10,SPEED_MAX_-9,SPEED_MAX_-8,SPEED_MAX_-7,SPEED_MAX_-6,SPEED_MAX_-5,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MAX_Y_0,SPEED_MAX_Y_1,SPEED_MAX_Y_2,SPEED_MAX_Y_3,SPEED_MIN_-10,SPEED_MIN_-9,SPEED_MIN_-8,SPEED_MIN_-7,SPEED_MIN_-6,SPEED_MIN_-5,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,SPEED_MIN_Y_0,SPEED_MIN_Y_1,SPEED_MIN_Y_2,SPEED_MIN_Y_3,N_VEHICLES_-10,N_VEHICLES_-9,N_VEHICLES_-8,N_VEHICLES_-7,N_VEHICLES_-6,N_VEHICLES_-5,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,N_VEHICLES_Y_0,N_VEHICLES_Y_1,N_VEHICLES_Y_2,N_VEHICLES_Y_3
0,0.0,337.0,2674.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199.6642,199.3150,204.7759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.4457,52.0643,43.0659,49.9930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263.734,292.858,294.476,312.274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,59.866,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,129.0,139.0,193.0,NaN,NaN,NaN
1,0.0,337.0,2734.0,1.0,2.0,1.0,6.0,Gestione_viabilita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.2940,21.7815,33.8448,35.4031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236.228,245.936,237.846,292.858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,63.102,59.866,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214.0,235.0,219.0,148.0
2,0.0,337.0,2795.0,1.0,2.0,1.0,6.0,Gestione_viabilita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.9124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.8807,39.7029,30.8512,28.1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352.724,286.386,343.016,318.746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,61.484,77.664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0,216.0,227.0,184.0,NaN
3,0.0,337.0,2796.0,1.0,2.0,1.0,6.0,Gestione_viabilita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.9124,201.9654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.8807,39.7029,30.8512,28.1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352.724,286.386,343.016,318.746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,61.484,77.664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0,216.0,227.0,184.0,NaN,NaN
4,0.0,337.0,2870.0,1.0,2.0,1.0,6.0,Gestione_viabilita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,213.5957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.7280,43.6813,43.5507,34.9842,33.4657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,334.926,328.454,351.106,286.386,362.432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,59.866,61.484,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,246.0,229.0,246.0,227.0,227.0
5,0.0,337.0,2980.0,1.0,2.0,1.0,2.0,Manutenzione_opere_in_verde,216.0995,218.6922,215.4648,214.3988,213.8432,214.4452,210.7883,213.6824,223.9108,223.1689,32.2995,29.4705,31.6018,27.5030,30.8558,32.2447,34.5211,33.1529,32.9331,30.7040,29.5927,28.9895,32.1558,29.6041,284.768,309.038,354.342,288.004,310.656,296.094,281.532,343.016,389.938,341.398,328.454,300.948,286.386,310.656,59.866,64.720,74.428,74.428,61.484,59.866,59.866,61.484,64.720,59.866,59.866,61.484,59.866,59.866,218.0,216.0,239.0,234.0,218.0,242.0,231.0,228.0,222.0,239.0,265.0,255.0,258.0,232.0
6,0.0,337.0,2981.0,1.0,2.0,1.0,2.0,Manutenzione_opere_in_verde,214.3988,213.8432,214.4452,210.7883,213.6824,223.9108,223.1689,218.5277,219.2802,214.6672,27.5030,30.8558,32.2447,34.5211,33.1529,32.9331,30.7040,29.5927,28.9895,32.1558,29.6041,34.4500,29.3056,27.2239,288.004,310.656,296.094,281.532,343.016,389.938,341.398,328.454,300.948,286.386,310.656,336.544,296.094,289.622,74.428,61.484,59.866,59.866,61.484,64.720,59.866,59.866,61.484,59.866,59.866,61.484,61.484,61.484,234.0,218.0,242.0,231.0,228.0,222.0,239.0,265.0,255.0,258.0,232.0,277.0,238.0,243.0
7,0.0,337.0,2982.0,1.0,2.0,1.0,2.0,Manutenzio

In [79]:
X['EVENT_DETAIL'] = X['EVENT_DETAIL'].fillna(0)
X['EVENT_TYPE'] = X['EVENT_TYPE'].fillna('Non presente')

In [80]:
X.fillna(0, inplace=True)
Y.fillna(0, inplace=True)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## Train/Validation split

In [81]:
from sklearn.model_selection import train_test_split

In [82]:
X, Y = shuffle(X, Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [83]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(27266, 71)
(6817, 71)
(27266, 4)
(6817, 4)


## Fit

In [92]:
from sklearn.metrics import mean_absolute_error

In [94]:
mae = []
for i,t in enumerate(Y.columns):
    print(f'{i}/{len(Y.columns)}')
    catboost = CatBoostRegressor(loss_function='MAE',
                                iterations=1000, depth=6, learning_rate=1,
                                cat_features=['EVENT_DETAIL','EVENT_TYPE','ROAD_TYPE']
                            )

    catboost.fit(X_train, y_train[t], verbose=False)

    y_pred = catboost.predict(X_test)

    mae.append(mean_absolute_error(y_test[t], y_pred))

0/4
1/4
2/4
3/4


In [95]:
mae

[4.0708849046662445, 4.119391535293603, 4.159952072952282, 4.276518655114229]

In [98]:
np.mean(mae)

4.156686792006589

In [43]:
model = MultiOutputRegressor(catboost)
model.fit(X=X, y=Y)

ValueError: could not convert string to float: 'Non presente'